<a href="https://colab.research.google.com/github/leeds1219/VQA_retriever/blob/main/MAML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install higher transformers torch torchvision

In [29]:
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

caption_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
caption_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

In [ ]:
image = Image.open("/content/sample.jpg")

In [ ]:
inputs = processor(images=image, return_tensors="pt", padding=True)
image_features = model.get_image_features(**inputs)

# print(image_features)

In [30]:
question = "Why do dogs wag their tails?"

In [31]:
text_inputs = processor(text=[question], return_tensors="pt", padding=True)
text_features = model.get_text_features(**text_inputs)

# print(text_features)

In [ ]:
captions = [
    "Dogs wag their tails to show happiness, excitement, or to get attention.",
    "The weather today is absolutely beautiful.",
    "I love ending the day with a good book.",
    "I long for peaceful moments in nature."
]

In [33]:
text_inputs = caption_processor(text=captions, return_tensors="pt", padding=True)
text_features = caption_model.get_text_features(**text_inputs)

# print(text_features)